In [1]:
import pandas as pd

In [2]:
patient_monthwise_revenue_test = pd.read_csv('data/patient_monthwise_revenue_test.csv')
patient_monthwise_revenue_train = pd.read_csv('data/patient_monthwise_revenue_train.csv')
physio_diagnosis_train = pd.read_csv('data/physio_diagnosis_train.csv')
physio_appts_train = pd.read_csv('data/physio_appts_train.csv')
patient_train_classified = pd.read_csv('data/patient_train_classified.csv')

/home/pradeep/softs/anaconda3/envs/portea/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/pradeep/softs/anaconda3/envs/portea/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
physio_diagnosis_train = physio_diagnosis_train.rename(columns={'patientId': 'patient_id'})
physio_appts_train = physio_appts_train.rename(columns={'pat_id': 'patient_id'})
patient_train_classified = patient_train_classified.rename(columns={'PID': 'patient_id'})

In [4]:
md_revenue_diagnosis_train = pd.merge(patient_monthwise_revenue_train, physio_diagnosis_train, on='patient_id')
md_revenue_diagnois_appts_train = pd.merge(md_revenue_diagnosis_train, physio_appts_train, on='patient_id')
md_revenue_diagnois_appts_class_train = pd.merge(md_revenue_diagnois_appts_train, patient_train_classified, on='patient_id')

In [5]:
final_raw_train = md_revenue_diagnois_appts_class_train.drop(columns=
    ['Unnamed: 20', 'Unnamed: 21','Unnamed: 7','Unnamed: 8',
     'serviceId', 'name','service_name','Service',
     'Appt_id', 'FVS', 'approx_age',
     'LVD_y', 'gender_y', 'revenue',
    'diagnosis_x'], axis=1)

In [6]:
final_raw_train = final_raw_train.rename(columns={'gender_x': 'gender', 'LVD_x':'LVD', 'diagnosis_y': 'diagnosis'})

In [7]:
final_raw_train.columns.values

array(['patient_id', 'visit_month_year', 'service_id', 'visits_count',
       'city', 'ref_type', 'ref_name', 'ref_source', 'FVD', 'FVM',
       'gender', 'LVD', 'brand', 'visits_required', 'avg_nps',
       'diagnosis', 'AGE', 'appt_id', 'startDateTime', 'status', 'cost',
       'discount', 'patient_ivr_confirmation', 'appointment_delayed',
       'Bucket', 'Revenue'], dtype=object)

In [8]:
final_raw_train.sample(5)

,patient_id,visit_month_year,service_id,visits_count,city,ref_type,ref_name,ref_source,FVD,FVM,...,AGE,appt_id,startDateTime,status,cost,discount,patient_ivr_confirmation,appointment_delayed,Bucket,Revenue
232452,32900,08/15,35,2,Mumbai,B2C,SEM (Google),Digital + ATL,18/09/2014,09/14,...,79,174917,2014-11-24 09:45:00,successful,0.0,0.0,\N,0,High,31200.0
441155,41249,12/14,2,23,Bangalore,B2C,None,Direct Organic,29/11/2014,11/14,...,63,224639,2015-01-06 17:30:00,successful,0.0,0.0,\N,0,High,49900.0
1687804,155602,10/16,2,17,Chennai,B2B,Doctor,Chemist + Doctors,14/01/2016,01/16,...,70,1569063,2016-06-09 14:00:00,successful,350.0,0.0,\N,0,High,177000.0
628898,54870,06/15,2,3,NCR,B2C,SEM (Google),Digital + ATL,25/02/2015,02/15,...,62,642707,2015-07-25 14:00:00,successful,0.0,0.0,\N,0,High,23300.0
2525649,276611,12/16,2,4,Chandigarh,B2C,Clinician (B2C),Referral,29/11/2016,11/16,...,0,2183055,2016-12-14 16:00:00,cancelled,500.0,50.0,\N,0,High-Med,5950.0


In [13]:
# Missing values
missing_values_count = final_raw_train.isnull().sum()

In [14]:
missing_values_count

patient_id                  0
visit_month_year            0
service_id                  0
visits_count                0
city                        0
ref_type                    0
ref_name                    0
ref_source                  0
FVD                         0
FVM                         0
gender                      0
LVD                         0
brand                       0
visits_required             0
avg_nps                     0
diagnosis                   0
AGE                         0
appt_id                     0
startDateTime               0
status                      0
cost                        0
discount                    0
patient_ivr_confirmation    0
appointment_delayed         0
Bucket                      0
Revenue                     0
dtype: int64

In [12]:
final_raw_train = final_raw_train.fillna(0)

In [15]:
final_raw_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2968558 entries, 0 to 2968557
Data columns (total 26 columns):
patient_id                  int64
visit_month_year            object
service_id                  int64
visits_count                int64
city                        object
ref_type                    object
ref_name                    object
ref_source                  object
FVD                         object
FVM                         object
gender                      object
LVD                         object
brand                       object
visits_required             int64
avg_nps                     object
diagnosis                   object
AGE                         object
appt_id                     int64
startDateTime               object
status                      object
cost                        float64
discount                    float64
patient_ivr_confirmation    object
appointment_delayed         int64
Bucket                      object
Revenue         